### Imports

In [1]:
import numpy
# prevent confusion
from lclPyO3 import lclPyO3 as lcl_rust

# termination functions
from lclPyO3.termination.min_temperature_termination_criterion import MinTemperatureTerminationCriterion
from lclPyO3.termination.max_seconds_termination_criterion import MaxSecondsTerminationCriterion

# simulated annealing
from lclPyO3.localsearch.simulatedannealing.simulated_annealing import SimulatedAnnealing
from lclPyO3.localsearch.simulatedannealing.geometric_cooling_function import GeometricCoolingFunction
from lclPyO3.localsearch.simulatedannealing.cnst_iterations_temp_function import CnstIterationsTempFunction

# steepest descent 
from lclPyO3.localsearch.steepestdescent.steepest_descent import SteepestDescent

# tabu search
from lclPyO3.localsearch.tabusearch.sum_diff_state import SumDiffState
from lclPyO3.localsearch.tabusearch.tabu_search import TabuSearch

# variable neighbourhood search
from lclPyO3.localsearch.vns.variable_neighbourhood import VariableNeighbourhood

# move functions
from lclPyO3.localsearch.move.tsp_array_swap import TspArraySwap
from lclPyO3.localsearch.move.array_reverse_order import ArrayReverseOrder
from lclPyO3.localsearch.move.multi_neighbourhood import MultiNeighbourhood

# evaluation function
from lclPyO3.evaluation.tsp_evaluation_function import TspEvaluationFunction

# Problem class
from lclPyO3.problem.array_problem import ArrayProblem

 #### rust setup TSP48

In [2]:
rEval=lcl_rust.Evaluation.tsp_from_dist_matrix("../data/distanceMatrix")

rMove0=lcl_rust.MoveType.swap_tsp()
rMove1=lcl_rust.MoveType.reverse()
rProblemSwap=lcl_rust.Problem.array_problem(rMove0,rEval)
rProblemReverse=lcl_rust.Problem.array_problem(rMove1,rEval)

#simulated annealing
rMinTemp=lcl_rust.Termination.min_temp(100)
rCooling=lcl_rust.Cooling.geometric_cooling(0.95)
rIter=lcl_rust.IterationsPerTemp.cnst_iter_temp(1500)
rSimAnn=lcl_rust.LocalSearch.simulated_annealing(2000,True,rProblemSwap,rMinTemp,rCooling,rIter)

#steepest descent
rMax5=lcl_rust.Termination.max_sec(5)
rSteepestDescent=lcl_rust.LocalSearch.steepest_descent(True,rProblemSwap,rMax5)

#tabu search
rTabuSearch=lcl_rust.LocalSearch.tabu_search(True,rProblemSwap,rMax5)

#vns
rMoveVNS=lcl_rust.MoveType.multi_neighbor([rMove0,rMove1])
rProblemVNS=lcl_rust.Problem.array_problem(rMoveVNS,rEval)
rVNS=lcl_rust.LocalSearch.vns(True,rProblemVNS,rMax5)

 #### python setup

In [3]:
pData=numpy.loadtxt("../data/distanceMatrix",dtype=float)

#simulated annealing
pMoveSA = TspArraySwap(pData.shape[0])
pEvalSA=TspEvaluationFunction(pData,pMoveSA)
pProblemSA=ArrayProblem(pEvalSA,pMoveSA,pData.shape[0])
pMinTemp = MinTemperatureTerminationCriterion(100)
pCooling = GeometricCoolingFunction(0.95)
pIter = CnstIterationsTempFunction(1500)
pSimAnn = SimulatedAnnealing(pProblemSA, pMinTemp, pCooling, pIter, logging=False, benchmarking=True)

#steepest descent
pMoveSD = TspArraySwap(pData.shape[0])
pEvalSD=TspEvaluationFunction(pData,pMoveSD)
pMax5SD=MaxSecondsTerminationCriterion(5)
pProblemSD=ArrayProblem(pEvalSD,pMoveSD,pData.shape[0])
pSteepestDescent=SteepestDescent(pProblemSD,pMax5SD,logging=False, benchmarking=True)

#Tabu Search
pMax5TB=MaxSecondsTerminationCriterion(5)
pMoveTB = TspArraySwap(pData.shape[0])
pEvalTB=TspEvaluationFunction(pData,pMoveTB)
pProblemTB=ArrayProblem(pEvalTB,pMoveTB,pData.shape[0])
pDiffState =SumDiffState()
pTabuSearch=TabuSearch(pProblemTB,pMax5TB,pDiffState,logging=False, benchmarking=True)

#vns
pMax5VNS=MaxSecondsTerminationCriterion(5)
pMove0 = TspArraySwap(pData.shape[0])
pMove1 = ArrayReverseOrder(pData.shape[0])
pMoveVNS = MultiNeighbourhood([pMove0,pMove0])

pEvalVNS=TspEvaluationFunction(pData,pMoveVNS)
pProblemVNS=ArrayProblem(pEvalVNS,pMoveVNS,pData.shape[0])
pVNS = VariableNeighbourhood(pEvalVNS,pMoveVNS,logging=False, benchmarking=True)


### Run tests

In [4]:
rustBenchRes=lcl_rust.benchmark([rProblemSwap,rProblemReverse],[rSimAnn,rSteepestDescent,rTabuSearch],rMax5,10)


In [5]:
print('____Rust Results___')
for alg in rustBenchRes:
    print('|--- Runs for algorithm ')
    for prob in alg:
        print('|--- Runs for seed ')
        for seed in prob:
            print(seed)
# pTime=pRes.data.time[-1:][0]
# rTime=rRes[-1:][0][0]/10**9
# print("Python time: "+ str(pTime))
# print("Rust time: "+ str(rTime))
# print("Procentuele versnelling: "+ str(((pTime-rTime)/pTime)*100))
# print("Aantal keer sneller: "+ str(pTime/rTime))


____Rust Results___
|--- Runs for algorithm 
|--- Runs for seed 
(5000246200, 36218.0, 36391.0, 31138063)
(5000001300, 37641.0, 37641.0, 31158027)
(5000001500, 36055.0, 36061.0, 32648012)
(5000001200, 35594.0, 35594.0, 32871227)
(5000001500, 35418.0, 35418.0, 33083807)
(5000001300, 34287.0, 34287.0, 31400704)
(5000001200, 35349.0, 35398.0, 32362621)
(5000001100, 36086.0, 36086.0, 32684038)
(5000001400, 36667.0, 36667.0, 32761289)
(5000001200, 36044.0, 36107.0, 32929614)
|--- Runs for seed 
(5000001600, 34349.0, 34349.0, 32550940)
(5000001400, 34071.0, 34071.0, 32610242)
(5000001800, 33809.0, 33809.0, 32795395)
(5000001400, 34540.0, 34540.0, 31795921)
(5000001400, 34281.0, 34281.0, 32038061)
(5000001200, 34078.0, 34078.0, 32562547)
(5000001500, 34420.0, 34562.0, 31759393)
(5000001200, 33581.0, 33581.0, 32630408)
(5000003000, 33970.0, 33970.0, 31788629)
(5000010800, 34234.0, 34234.0, 32438214)
|--- Runs for algorithm 
|--- Runs for seed 
(1181000, 55658.0, 55658.0, 42)
(1423700, 55658.0,